## Importing libraries

In [ ]:
import pandas
import keras

In [ ]:
from keras.layers import Dense, Dropout, Activation,LSTM
from keras.layers import Embedding
from keras.layers import GRU
from keras.layers import Conv1D, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
finaldata=pd.read_csv('/content/drive/My Drive/project/data.csv')

In [ ]:
finaldata['target']=finaldata['target'].replace({'NoHate':0,'Hate':1})

In [ ]:
finaldata.shape

(187902, 11)

In [ ]:
finaldata.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'comment_text', 'target', 'Num_words',
       'words', 'clean_comment', 'clean_comment_wo_stop',
       'clean_comment_lemmatized', 'clean_comment_string',
       'clean_comment_string1'],
      dtype='object')

In [ ]:
finaldata.sample(2)

,Unnamed: 0.1,Unnamed: 0,comment_text,target,Num_words,words,clean_comment,clean_comment_wo_stop,clean_comment_lemmatized,clean_comment_string,clean_comment_string1
81000,81000,81000,New Jersey Devils and Detroit Red Wings of 199...,0,49,"['New', 'Jersey', 'Devils', 'and', 'Detroit', ...",new jersey devils and detroit red wings of ...,new jersey devils detroit red wings dont know ...,new jersey devils detroit red wings dont know ...,new jersey devils detroit red wings dont know ...,new jersey devils detroit red wings dont know ...
17891,17891,17891,"""\nArianit, I started the RM and btw WW I hope...",0,28,"['""', 'Arianit,', 'I', 'started', 'the', 'RM',...",arianit i started the rm and btw ww i hope ...,arianit started rm btw ww hope take three repe...,arianit started rm btw ww hope take three repe...,arianit started rm btw ww hope take three repe...,arianit started rm btw ww hope take three repe...


In [ ]:
finaldata = finaldata.dropna()

In [ ]:
textdata=finaldata['clean_comment_string1']
target=finaldata['target']

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(textdata,target,test_size=0.25,random_state=12,stratify=target)

In [ ]:
X_train.shape

(140756,)

In [ ]:
Y_train.value_counts()

0    118402
1     22354
Name: target, dtype: int64

In [ ]:
X_train1=X_train
Y_train1=Y_train
X_test1=X_test
Y_test1=Y_test

In [ ]:
Y_train

99249     0
112221    0
184367    1
100906    0
70145     0
         ..
2490      1
117913    0
13266     0
144583    0
45754     1
Name: target, Length: 140756, dtype: int64

In [ ]:
X_train1

99249                     agree disagree deletion ok cheers
112221    european hornet added european hornet repeat r...
184367    lot people point say trump stupid things trump...
100906                   link previous discussion show diff
70145     adverts lol added sentence television section ...
                                ...                        
2490      fuck wrong people stop critizing us writing op...
117913       paring list asking people put x next ones like
13266                                     also better beter
144583    boethius definition catholic encyclopedia refe...
45754     ugly fact ugly people put image suppose lie te...
Name: clean_comment_string1, Length: 140756, dtype: object

In [ ]:
def tokenizer(X_train, X_test,maxword=75000,sequ=50):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0).astype('str')
    text = np.array(text)
    tokenizer = Tokenizer(num_words=maxword)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=sequ)
    indices = np.arange(text.shape[0])
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("/content/drive/My Drive/project/glove.6B.50d.txt", encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    return (X_train, X_test, word_index,embeddings_index)

In [ ]:
train_Glove,test_Glove, wordindex,embeddingsindex = tokenizer(X_train,X_test)

(187675, 50)


In [ ]:
def LSTMCNN(wordi, embeddings_index, nclasses, MAX_SEQUENCE_LENGTH=50, EMBEDDING_DIM=50):
    kernel_size = 2
    filters = 64
    pool_size = 2
    gru_node = 256
    embedding_matrix = np.random.random((len(wordi) + 1, EMBEDDING_DIM))
    for word, i in wordi.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            if len(embedding_matrix[i]) !=len(embedding_vector):
                print("EMBEDDING_DIM is not equal to embedding_vector file ,GloVe,")
                exit(1)

            embedding_matrix[i] = embedding_vector
    model = Sequential()
    model.add(Embedding(len(wordi) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Conv1D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, return_sequences=True, recurrent_dropout=0.2))
    model.add(LSTM(gru_node, recurrent_dropout=0.2))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(nclasses))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
modelrcnnlstm = LSTMCNN(wordindex,embeddingsindex, 3)

In [ ]:
modelrcnnlstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            8843350   
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 49, 64)            6464      
                                                                 
 max_pooling1d (MaxPooling1  (None, 24, 64)            0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 64)            8256      
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 11, 64)            0         
 g1D)                                                   

In [ ]:
modelrcnnlstm.fit(train_Glove,Y_train,epochs=5,batch_size=128,verbose=2)

Epoch 1/5
1100/1100 - 138s - loss: 0.3163 - accuracy: 0.8742 - 138s/epoch - 125ms/step
Epoch 2/5
1100/1100 - 66s - loss: 0.2603 - accuracy: 0.8940 - 66s/epoch - 60ms/step
Epoch 3/5
1100/1100 - 60s - loss: 0.2439 - accuracy: 0.8999 - 60s/epoch - 55ms/step
Epoch 4/5
1100/1100 - 60s - loss: 0.2303 - accuracy: 0.9052 - 60s/epoch - 54ms/step
Epoch 5/5
1100/1100 - 60s - loss: 0.2196 - accuracy: 0.9105 - 60s/epoch - 55ms/step


In [ ]:
modelrcnnlstm.save('/content/drive/My Drive/project/rcnnlstm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
hate_prediction=modelrcnnlstm.predict(test_Glove)

1467/1467 [==============================] - 10s 6ms/step


In [ ]:
predict_class = np.argmax(hate_prediction, axis=1)

In [ ]:
import tensorflow as tf

In [ ]:
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/project/rcnnlstm.h5')

BERT TOEKNIZER

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

In [ ]:
berttokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
#Specify maximum sequence length to pad or truncate
max_len = 0

for seq in X_train:
  if seq:
    # Tokenize the text by BERT tokenizer
    input_ids = berttokenizer.encode(seq[:511], add_special_tokens=True)
    max_len = max(max_len, len(input_ids))


print('Maximum sequence length', max_len)


Maximum sequence length 258


In [ ]:
# Tokenize all of the sequences and map the tokens to thier IDs.
input_ids_train = []
attention_masks_train = []

# For every sequences
for seq in X_train1:
    encoded_dict = berttokenizer.encode_plus(
                        seq,                             # Sequence to encode
                        add_special_tokens = True,       # Add '[CLS]' and '[SEP]'
                        max_length = 128,
                        padding = 'max_length',          # Pad and truncate
                        truncation=True,                 #Truncate the seq
                        return_attention_mask = True    # Construct attn. masks
                                 # Return pytorch tensors
                   )

    # Add the encoded sequences to the list
    input_ids_train.append(encoded_dict['input_ids'])

    # And its attention mask
    attention_masks_train.append(encoded_dict['attention_mask'])

In [ ]:
# Tokenize all of the sequences and map the tokens to thier IDs.
input_ids_test = []
attention_masks_test = []

# For every sequences
for seq in X_test1:
    encoded_dict_test = berttokenizer.encode_plus(
                        seq,                             # Sequence to encode
                        add_special_tokens = True,       # Add '[CLS]' and '[SEP]'
                        max_length = 128,
                        padding = 'max_length',          # Pad and truncate
                        truncation=True,                 #Truncate the seq
                        return_attention_mask = True    # Construct attn. masks
                                 # Return pytorch tensors
                   )

    # Add the encoded sequences to the list
    input_ids_test.append(encoded_dict_test['input_ids'])

    # And its attention mask
    attention_masks_test.append(encoded_dict['attention_mask'])

In [ ]:
import tensorflow as tf

In [ ]:
input_ids=tf.convert_to_tensor(input_ids_train)
attention_masks=tf.convert_to_tensor(attention_masks_train)

In [ ]:
input_ids_test=tf.convert_to_tensor(input_ids_test)
attention_masks_test=tf.convert_to_tensor(attention_masks_test)

In [ ]:
train_target=tf.convert_to_tensor(Y_train1)

In [ ]:
test_target=tf.convert_to_tensor(Y_test1)

In [ ]:
input_ids.shape

TensorShape([140756, 128])

In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification

In [ ]:
bert_model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model_path="/content/drive/My Drive/project/bert_model.h5"
log_path="/content/drive/My Drive/project"

In [ ]:
callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath=model_path,save_weights_only=True,monitor="val_loss",mode="min",save_best_only=True),tf.keras.callbacks.TensorBoard(log_dir=log_path)]
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

In [ ]:
bert_model.compile(loss=loss,optimizer=optimizer,metrics=[metrics])

In [46]:
history=bert_model.fit([input_ids,attention_masks],train_target,batch_size=32,epochs=2,validation_data=([input_ids_test,attention_masks_test],test_target),callbacks=callbacks)

Epoch 1/2
4399/4399 [==============================] - 3945s 887ms/step - loss: 0.1786 - accuracy: 0.9256 - val_loss: 0.2323 - val_accuracy: 0.9226
Epoch 2/2
4399/4399 [==============================] - 3865s 879ms/step - loss: 0.1408 - accuracy: 0.9409 - val_loss: 0.2147 - val_accuracy: 0.9238


LIME